In [29]:
import os
from selenium import webdriver
import PIL
import requests
searches_txt = open('searches.txt', 'r')
DRIVER = 'chromedriver'

In [25]:
#make a directory if it doesnt exist for each of the searches in .txt
searches_arr = []
for line in searches:
    print(line)
    search = line.replace('\n', '')
    print(search)
    searches_arr.append(search)
    if not os.path.exists(search):
        os.makedirs(search)
    else:
        print('Directory exists')

In [8]:
wd = webdriver.Chrome(executable_path=DRIVER)
wd.get('https://google.com')

In [9]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')